# Install basic requirements

In [ ]:
pip install -U whylogs pandas

In [2]:
import whylogs
import pandas as pd

# Load example data

The example data is prepared from GitHub. You can use your own data if you want.

WhyLabs only requires whylogs API - your raw data never leaves your premise

In [3]:
full_data = pd.read_csv("https://github.com/whylabs/whylogs-examples/raw/4c52209fc5aca3a5a462242db2f1520452368670/python/lending_club_demo.csv")

full_data.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,1.643400e+04,0.0,16434.000000,16434.00000,16434.000000,16434.000000,16434.000000,1.643400e+04,16429.000000,16434.000000,...,53.0,53.000000,53.0,53.000000,46.000000,53.000000,53.000000,120.000000,120.000000,120.000000
mean,6.032493e+07,NaN,14835.055373,14827.79299,14806.153567,13.219376,442.485652,7.769430e+04,18.546474,0.338627,...,3.0,128.221321,3.0,12.679245,364.672174,10205.598679,183.086981,5540.947917,48.885917,10.366667
std,3.624241e+07,NaN,8864.324654,8861.86405,8866.243509,4.695684,261.004530,7.661281e+04,9.347356,0.909076,...,0.0,103.278475,0.0,9.549010,286.708985,7498.442069,175.239316,3958.539886,9.316292,9.125579
min,8.804600e+04,NaN,1000.000000,1000.00000,0.000000,5.320000,30.160000,0.000000e+00,0.000000,0.000000,...,3.0,8.000000,3.0,0.000000,52.890000,845.360000,0.010000,340.860000,19.590000,0.000000
25%,2.911401e+07,NaN,8000.000000,8000.00000,8000.000000,9.760000,253.187500,4.600000e+04,12.140000,0.000000,...,3.0,54.030000,3.0,0.000000,164.062500,5018.900000,46.760000,2887.500000,45.000000,1.000000
50%,6.418557e+07,NaN,12800.000000,12800.00000,12800.000000,12.740000,379.760000,6.500000e+04,17.910000,0.000000,...,3.0,96.540000,3.0,15.000000,265.455000,7677.930000,117.690000,4491.405000,45.000000,11.000000
75%,9.128316e+07,NaN,20000.000000,20000.00000,20000.000000,15.990000,588.080000,9.200000e+04,24.370000,0.000000,...,3.0,168.670000,3.0,20.000000,493.147500,12926.810000,326.930000,7557.495000,50.790000,18.000000
max,1.201175e+08,NaN,40000.000000,40000.00000,40000.000000,30.990000,1618.030000,5.499500e+06,281.330000,21.000000,...,3.0,507.430000,3.0,27.000000,1522.290000,30158.880000,659.410000,25000.000000,91.200000,24.000000


# Creating batches of data

WhyLogs handles profiles in batches. What that means is:
* If two profiles are sent for the same day, we merge them into one profile (they are mergeable!!)
* To view profiles across different batches, you need to break data into smaller batches


**NOTE**: profiles will show up in real time if they are dated within the last 7 days.

Here we are creating 8 batches

In [4]:
# Create a list of dates
dates = ['Jan-2017', 'Feb-2017', 'Mar-2017', 'Apr-2017', 'May-2017', 'Jun-2017', 'Jul-2017', 'Aug-2017']

dates.reverse()
print(dates)

['Aug-2017', 'Jul-2017', 'Jun-2017', 'May-2017', 'Apr-2017', 'Mar-2017', 'Feb-2017', 'Jan-2017']


In [5]:
pdfs = []  # list with original profile

for date in dates:
    subset_data = full_data[full_data['issue_d']==date]
    pdfs.append(subset_data)

# Configure whylogs

whylogs, by default, does not send information to WhyLabs.

There are a few small steps you need to set up. If you haven't got the access key, please onboard with WhyLabs

In [6]:
from whylogs.app import Session
from whylogs.app.writers import WhyLabsWriter
import os
import datetime

In [7]:
import getpass

# Using development environment. Not needed normally
os.environ["WHYLABS_API_ENDPOINT"] = "https://songbird.development.whylabsdev.com"

# set your org-id here
print("Enter your WhyLabs Org ID")
os.environ["WHYLABS_DEFAULT_ORG_ID"] = input()
# set your API key here
print("Enter your WhyLabs API key")
os.environ["WHYLABS_API_KEY"] = getpass.getpass()
print("Using API Key ID: ", os.environ["WHYLABS_API_KEY"][0:10])

Enter your WhyLabs Org ID


 org-5953


Enter your WhyLabs API key


 ································································


Using API Key ID:  Em2W1PzHqc


## Creating session

Once the environments are set, let's create a whylogs session with a WhyLabs writer.

Note that you can add your local writer or S3 writer if you want here. Check out the API docs for more information.

In [8]:
# create WhyLabs session
writer = WhyLabsWriter("", formats=[])
session = Session(project="demo-project", pipeline="demo-pipeline", writers=[writer])

## Logging to WhyLabs

Ensure you have a **model ID** (also called **dataset ID**) before you start!

In [ ]:
print("Enter your model ID from WhyLabs:")
model_id = input()
for i, df in enumerate(pdfs):
    # walking backwards. Each dataset has to map to a date to show up as a different batch
    # in WhyLabs
    dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)
    
    # Create new logger for date
    with session.logger(tags={"datasetId": model_id}, dataset_timestamp=dt) as ylog:
        print("Log data frame for ", dt)
        ylog.log_dataframe(df)

## Voila

* Now check the application to see if your **statistics** are in!!
* Also, run the above cell again for the same model ID, do you see the statistics changes in WhyLabs? Especially the counters?